In [2]:
import os

In [3]:
%pwd

'd:\\voting_application\\Text_summariser-NLP-\\research'

In [4]:
os.chdir("../")

In [5]:
%pwd

'd:\\voting_application\\Text_summariser-NLP-'

In [ ]:
from dataclasses import dataclass
from pathlib import Path 

@dataclass(frozen = TRUE)
class ModelEvalutionConfig:
    root_dir : Path
    data_path: Path
    model_path: Path
    tokenizer_path: Path
    metric_file_name: Path 

In [ ]:
from text_summarizer.constants import *
from text_summarizer.utlis.common import  read_yaml, create_directories

In [ ]:
class ConfigurationManager:
    def __init__(
            self,
            config_filepath = CONFIG_FILE_PATH,
            params_filepath = PARAMS_FILE_PATH):

            self.config = read_yaml(config_filepath)
            self.params = read_yaml(params_filepath)
            create_directories([self.config.artifacts_root])
            
    def get_model_evalution_config(self) -> ModelEvalutionConfig:
          config =  self.config.model_evalution_config

          create_directories([config.root_dir])
          model_evalution_config = ModelEvalutionConfig(
                root_dir=config.root_dir,
                data_path= config.data_path,
                model_path= config.model_path,
                tokenizer_path= config.tokenizer_path,
                metric_file_name= config.metric_file_name)
          return model_evalution_config

In [ ]:
from transformers import AutoModelForSeq2Seq , AutoTokenizer
from datasets import load_dataset , load_from_disk
import troch
import pandas as pd
from tqdm import tqdm 


In [ ]:
class ModelEvalution:
    def __init__(self, config: ModelEvalutionConfig):
        self.config = config
    
    def generate_batch_size(self, list_of_element, batch_size):
        """ split the dataset into smaller batches that we process simultanes vield  succusive batch-sized chunks from list_of_element """
        for i in range(0, len(list_of_element), batch_size):
            yield list_of_element[i:i + batch_size]
    
    def calculate_metric_on_test_ds(self, dataset,metric, model,tokenizer, batch_size = 16 , device  = "cuda" if torch.cuda.is_available() else "cpu "
                                    batch_size = 16 , device = "cuda" if torch.cuda.is_  ):
                        